In [63]:
from Bio.SeqIO.QualityIO import FastqGeneralIterator
from Bio import SeqIO
import pandas as pd
import numpy as np

In [64]:
# 读取sgRNA_reference，生成表格“名字+序列”
file_handle = open('sgRNA_reference.fa','r')
sgRNA_reference = file_handle.readlines()

# 读取名字
sgRNA_reference_2 = sgRNA_reference[::2]
sgRNA_name = []
for sgRNA in sgRNA_reference_2:
    sgRNA = sgRNA.strip().replace('>','')
    sgRNA_list = sgRNA.split('_')
    sgRNA = sgRNA_list[0]
    sgRNA_name.append(sgRNA)

#读取序列
sequencesrna = []
for record in SeqIO.parse('sgRNA_reference.fa','fasta'):
    sequencesrna.append(str(record.seq))

#生成表格
a = {'sequencesrna':sequencesrna}
sgRNA_reference_pd = pd.DataFrame(a,index = sgRNA_name)
sgRNA_reference_pd 

,sequencesrna
A2M,TAGGAAGGCTTTGCATATT
A2M,TACTCCCTACAATCCATCT
A2M,ACAATCCATCTGGTCCCAA
A2M,ATGATACTGTCATCTTCTT
A2M,ACAGACGCCTCAGTCTCTG
...,...
non-targeting,CGTTTTTGCCCCCGATAAG
non-targeting,CCGTAGCGGCGTAACGCGG
non-targeting,GAGTCTTGATCTGCTCCCA
non-targeting,GCGAGGGTGTCCTCCCATG


In [75]:
#读入t 0时刻文件
sequences0 = []
for title, seq, qual in FastqGeneralIterator('t0.fastq'):
    #数据清理
    sequences0.append(seq[20:39])

#计算t 0时刻频数 
counts0=pd.Series(sequences0).value_counts()
countsd0 = pd.DataFrame(counts0,columns=['counts0'])
df_merge = sgRNA_reference_pd.merge(countsd0,left_on = 'sequencesrna', right_index = True)
df_merge['freq0']=df_merge['counts0']/df_merge['counts0'].sum()

#读入t 10时刻文件
sequences10 = []
for title, seq, qual in FastqGeneralIterator('t10.fastq'):
    sequences10.append(seq[20:39])

#计算t 10时刻频数
counts10=pd.Series(sequences10).value_counts()
countsd10 = pd.DataFrame(counts10,columns=['counts10'])
df_merge = df_merge.merge(countsd10,left_on = 'sequencesrna', right_index = True)
df_merge['freq10']=df_merge['counts10']/df_merge['counts10'].sum()

#计算变化率
df_merge['log2f']=np.log2(df_merge['freq10']/df_merge['freq0'])

In [74]:
# 计算 median of control sgRNAs
median_of_control_sgRNAs = np.median(df_merge.loc['non-targeting']['log2f'])
df_merge['log2f - median_of_control'] = df_merge['log2f'] - median_of_control_sgRNAs
df_merge

,sequencesrna,counts0,freq0,counts10,freq10,log2f,log2f - median_of_control
A2M,TAGGAAGGCTTTGCATATT,304,0.000095,310,0.000097,0.027794,0.004350
A2M,TACTCCCTACAATCCATCT,409,0.000127,381,0.000119,-0.102713,-0.126157
A2M,ACAATCCATCTGGTCCCAA,194,0.000060,180,0.000056,-0.108463,-0.131906
A2M,ATGATACTGTCATCTTCTT,521,0.000162,469,0.000146,-0.152098,-0.175542
A2M,ACAGACGCCTCAGTCTCTG,368,0.000115,386,0.000120,0.068492,0.045048
...,...,...,...,...,...,...,...
non-targeting,CGTTTTTGCCCCCGATAAG,181,0.000056,143,0.000045,-0.340377,-0.363821
non-targeting,CCGTAGCGGCGTAACGCGG,160,0.000050,147,0.000046,-0.122659,-0.146102
non-targeting,GAGTCTTGATCTGCTCCCA,149,0.000046,139,0.000043,-0.100630,-0.124074
non-targeting,GCGAGGGTGTCCTCCCATG,296,0.000092,311,0.000097,0.070915,0.047471


In [76]:
df_merge.to_csv('result2.csv')